In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# hyperparamters
SHEET_PATH = 'TT Optimization.xlsx'
MAX_GAP = 7
MIN_GAP = 2
MAX_TIMESLOT_DATE = 8
N_COURSES = 10

In [4]:
df_students = pd.read_excel(SHEET_PATH, sheet_name="Students")
df_courses = pd.read_excel(SHEET_PATH, sheet_name="Courses", index_col="#")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [4]:
df_students[[(list(df_students.columns)).pop()]].head()

,Concat
0,"1, 9, 8, 5, 7, 6, 2"
1,"1, 5, 6, 9, 4, 2,"
2,"7, 9, 5, 2, 8, 1,"
3,"7, 5, 1, 8, 6, 2, 9"
4,"1, 8, 3, 2, 5, 6,"


In [5]:
students_taking_it = df_students.melt(id_vars=['Name'],
                                      value_vars=['C1', 'C2', 'C3', 'C4', 'C5', 'C6'])\
                                      .groupby('value').count()\
                                      .reset_index()\
    .rename(columns={'variable': 'Course', 'Name': 'students_taking_it'})[['Course', 'students_taking_it']]

In [191]:
df_courses

,Name,Code,Dept,ID,CH,Score,CW,Factor,recommended_gap
#,,,,,,,,,
1,Signals,ECE251,ECE,251,4,2.5,55,3.2,7.000000
2,HR,ASU111,ASU,111,3,3.3,53,2.1,4.962963
3,EDA,CSE312,CSE,312,2,3.7,58,0.5,2.000000
4,DSA,CSE331,CSE,331,3,3.5,56,1.3,3.481481
5,Algorithms,CSE332,CSE,332,3,3.2,54,2.0,4.777778
6,DB,CSE333,CSE,333,3,3.5,53,1.9,4.592593
7,Testing,CSE338,CSE,338,3,3.7,57,0.9,2.740741
8,Compilers,CSE439,CSE,439,3,3.5,55,1.5,3.851852
9,AI,CSE472,CSE,472,3,3.5,57,1.1,3.111111


In [33]:
def map_courses_to_indices(courses):
    ret = set()
    for course in courses:
        ret.add(int(course))
    return ret

In [34]:
course_dict = {}
for index, row in df_students.iterrows():
    courses_of_student = set(row["Concat"].split(", "))
    try:
        courses_of_student.remove("")
    except:
        pass
    course_dict[index] = (map_courses_to_indices(courses_of_student))

In [35]:
course_dict  # Student ID : [*course_ids]

{0: {1, 2, 5, 6, 7, 8, 9},
 1: {1, 2, 4, 5, 6, 9},
 2: {1, 2, 5, 7, 8, 9},
 3: {1, 2, 5, 6, 7, 8, 9},
 4: {1, 2, 3, 5, 6, 8},
 5: {2, 3, 5, 6, 9},
 6: {1, 2, 5, 6, 7, 8, 9},
 7: {1, 2, 4, 5, 6, 7, 9},
 8: {1, 2, 4, 6, 7, 8, 9},
 9: {1, 2, 4, 5, 6, 7}}

In [36]:
list(course_dict.values())

[{1, 2, 5, 6, 7, 8, 9},
 {1, 2, 4, 5, 6, 9},
 {1, 2, 5, 7, 8, 9},
 {1, 2, 5, 6, 7, 8, 9},
 {1, 2, 3, 5, 6, 8},
 {2, 3, 5, 6, 9},
 {1, 2, 5, 6, 7, 8, 9},
 {1, 2, 4, 5, 6, 7, 9},
 {1, 2, 4, 6, 7, 8, 9},
 {1, 2, 4, 5, 6, 7}]

In [37]:
def get_conflict_matrix(course_dict):
    course_list = list(course_dict.values())

    # Create a list of unique course IDs
    course_ids = sorted(set.union(*course_list))

    # Initialize the conflict matrix to 1 for all courses
    conflict_matrix = [[1] * len(course_ids) for _ in range(len(course_ids))]

    # Iterate over each pair of courses and check if they appear together in any student's course set
    for i, course1 in enumerate(course_ids):
        for j, course2 in enumerate(course_ids):
            if i < j:
                for courses in course_list:
                    if course1 in courses and course2 in courses:
                        conflict_matrix[i][j] = 0
                        conflict_matrix[j][i] = 0
                        break
    return conflict_matrix

In [38]:
conflict_matrix = get_conflict_matrix(course_dict)
print("    ", *[f"{i} " for i in range(1, 10)])
# Print the conflict matrix
for i, row in enumerate(conflict_matrix):
    print(i+1, ":", row)

     1  2  3  4  5  6  7  8  9 
1 : [1, 0, 0, 0, 0, 0, 0, 0, 0]
2 : [0, 1, 0, 0, 0, 0, 0, 0, 0]
3 : [0, 0, 1, 1, 0, 0, 1, 0, 0]
4 : [0, 0, 1, 1, 0, 0, 0, 0, 0]
5 : [0, 0, 0, 0, 1, 0, 0, 0, 0]
6 : [0, 0, 0, 0, 0, 1, 0, 0, 0]
7 : [0, 0, 1, 0, 0, 0, 1, 0, 0]
8 : [0, 0, 0, 0, 0, 0, 0, 1, 0]
9 : [0, 0, 0, 0, 0, 0, 0, 0, 1]


In [62]:
def get_conflict_dict(conflict_matrix):
    conflict_dict = {}
    for i, row in enumerate(conflict_matrix):
        course = i+1
        conflict_set = set(j+1 for j, val in enumerate(row) if val == 0 or j == i)
        conflict_dict[course] = conflict_set

    return conflict_dict

In [63]:
conflict_dict = get_conflict_dict(conflict_matrix)
conflict_dict

{1: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 2: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 3: {1, 2, 3, 5, 6, 8, 9},
 4: {1, 2, 4, 5, 6, 7, 8, 9},
 5: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 6: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 7: {1, 2, 4, 5, 6, 7, 8, 9},
 8: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 9: {1, 2, 3, 4, 5, 6, 7, 8, 9}}

In [64]:
def get_freq(course_dict):
    course_freq = {course: sum(course in courses for courses in course_dict.values())
                   for course in set.union(*course_dict.values())}
    return sorted(list(course_freq.items()),
                  key=lambda x: -x[1])

In [65]:
get_freq(course_dict)

[(2, 10), (1, 9), (5, 9), (6, 9), (9, 8), (7, 7), (8, 6), (4, 4), (3, 2)]

In [66]:
# To normalize the factors
min_factor = float(df_courses["Factor"].min())
max_factor = float(df_courses["Factor"].max())

In [67]:
def gamma(factor):
    return (factor - min_factor) / (max_factor - min_factor) * (MAX_GAP - MIN_GAP) + MIN_GAP

In [68]:
# Apply gamma to factor
df_courses['recommended_gap'] = df_courses["Factor"].apply(gamma)

$$\Gamma(DepartmentFactor * CreditHours * ((1-\alpha) \cdot AverageCreditsLost + \alpha \cdot \Psi(AverageCourseworkMarksLost)))$$

$$BestGapFactor(C) = \Gamma(DepartmentFactor * CreditHours * ((1-\alpha) \cdot AverageCreditsLost + \alpha \cdot \Psi(AverageCourseworkMarksLost)))$$

In [188]:
def generate_feasible_timetable(course_dict):
    # Create a conflict dictionary and a frequency dictionary
    conflict_matrix = get_conflict_matrix(course_dict)
    conflict_dict = get_conflict_dict(conflict_matrix)
    freq_dict = get_freq(course_dict)
    
    def check_if_can_append_to_timeslot(min_timeslot, course_id, non_conflicts):
        for course_id_inside in min_timeslot:
            if course_id_inside not in non_conflicts:
                return False
        return True
    
    # Create a list of course IDs sorted by decreasing frequency
    course_ids = [x[0] for x in freq_dict]
    # Initialize an empty list of timeslots
    timeslots = []

    # Iterate over each course ID in the sorted list
    for course_id in course_ids:
        # Find the non-conflicting courses for this course
        non_conflicts = set(course_ids) - conflict_dict[course_id]
        # If there are no non-conflicting courses, create a new timeslot
        if not non_conflicts:
            timeslots.append([course_id])
        else:
            # Find the timeslot with the fewest courses
            sorted_indices = sorted(range(len(timeslots)), key=lambda i: len(timeslots[i]))
            min_timeslot = min(timeslots, key=lambda x: len(x))
            has_appended = False
            for idx in sorted_indices:
                # Check if adding this course to the timeslot would cause a clash
                can_append = check_if_can_append_to_timeslot(timeslots[idx], course_id, non_conflicts)
                if can_append:
                    # Add this course to the timeslot with the fewest courses
                    has_appended = True
                    timeslots[idx].append(course_id)
                    break
            if not has_appended:
                # Create a new timeslot if there is a clash
                timeslots.append([course_id])
        
        print(course_id, "\t\t", *non_conflicts, "\t\t", timeslots)

    return timeslots

In [190]:
timetable = generate_feasible_timetable(course_dict)
timetable

2 		 		 [[2]]
1 		 		 [[2], [1]]
5 		 		 [[2], [1], [5]]
6 		 		 [[2], [1], [5], [6]]
9 		 		 [[2], [1], [5], [6], [9]]
7 		 3 		 [[2], [1], [5], [6], [9], [7]]
8 		 		 [[2], [1], [5], [6], [9], [7], [8]]
4 		 3 		 [[2], [1], [5], [6], [9], [7], [8], [4]]
3 		 4 7 		 [[2], [1], [5], [6], [9], [7, 3], [8], [4]]


[[2], [1], [5], [6], [9], [7, 3], [8], [4]]

In [126]:
conflict_dict

{1: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 2: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 3: {1, 2, 3, 5, 6, 8, 9},
 4: {1, 2, 4, 5, 6, 7, 8, 9},
 5: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 6: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 7: {1, 2, 4, 5, 6, 7, 8, 9},
 8: {1, 2, 3, 4, 5, 6, 7, 8, 9},
 9: {1, 2, 3, 4, 5, 6, 7, 8, 9}}

In [136]:
def verify_timetable(timetable, conflict_dict):
    for timeslot in timetable:
        for i, course1 in enumerate(timeslot):
            for j, course2 in enumerate(timeslot):
                if i != j:
                    if course2 in conflict_dict[course1]:
                        return False
    return True

In [137]:
for tt in [tts]:
    print(verify_timetable(tt, conflict_matrix))

False


$$\Lambda(x, y) = (GapFactor(x) - (T(x) - T(y))^2$$

In [ ]:
# def compute_loss(curr_course, prev_course):
#     # If we are testing against the first subject,
#     # find the difference between it and the first
#     # possible timeslot
#     if prev_course is None:
#         return curr_course["Date"] ** 2
    
#     # Otherwise compute the loss normally
#     return (curr_course["recommended_gap"] - (curr_course["Date"] - prev_course["Date"])) ** 2

$$J(S)=\sum_{i=0}^{S-1}\sum_{j=0}^{C(S_i)-1} \Lambda(C(S_i)_j, C(S_i)_{j-1})$$

In [ ]:
# def compute_cost(courses_per_student):
#     """Computes the cost for the given students df"""
#     cost = 0
    
#     # Loop through each student
#     for name, courses_of_that_student in courses_per_student.items():
#         # Sort the courses taken by that student by date
#         courses = courses_per_student[name].sort_values(by="Date")
        
#         # Loop through each subject taken by that student
#         loss = 0
#         for j in range(0, len(courses)):
#             curr_course = courses.iloc[j]
#             if j==0:
#                 prev_course = None
#             else:
#                 prev_course = courses.iloc[j-1]
#             loss += compute_loss(curr_course, prev_course)
#         cost += loss
#     return cost